In [1]:
import csv
import collections

from argparse import ArgumentParser
from sklearn.model_selection import train_test_split
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

In [2]:
!jupyter nbconvert --to script "BERT Kaggle Notebook.ipynb"

[NbConvertApp] Converting notebook BERT Kaggle Notebook.ipynb to script
[NbConvertApp] Writing 5506 bytes to BERT Kaggle Notebook.py


In [2]:
SPLIT = .15

In [3]:
labels = []
texts = []

# read full dataset file
with open("../articles.csv",  "r", encoding='utf-8',) as csvfile:
    reader = csv.reader(csvfile, delimiter=';', quotechar='\'')
    for row in reader:
        labels.append(row[0])
        texts.append(row[1])

tr_Df=pd.concat([pd.Series(texts, name="Text"), pd.Series(labels,name='Label')], axis=1)
# split dataset
X_train, X_test, y_train, y_test = train_test_split( tr_Df['Text'], tr_Df['Label'], test_size=SPLIT, random_state=42,stratify=tr_Df['Label'])

# write train and test datasets
train=pd.DataFrame(pd.concat([X_train,y_train],axis=1))
test=pd.DataFrame(pd.concat([X_test,y_test],axis=1))


In [4]:
train

,Text,Label
9321,Die Energiewirtschaft hat ihre Strategie bis 2...,Wirtschaft
5703,"Östereich siegte in Podgorica zuerst gegen 12,...",Sport
8630,Sagis Vertreter Phillip Burns und Barry Gilber...,Wirtschaft
44,Keine offizielle Bestätigung über Verhandlungs...,Etat
537,"Roland Düringer in Autorevue TV, vom Leiden de...",Etat
...,...,...
2184,Auch Einreise- und Vermögenssperren gegen Luka...,International
5236,In der chinesischen Hauptstadt fahren Österrei...,Panorama
220,"Strache will weltoffen sein, atmete man in dem...",Etat
5186,25-Jähriger wollte TV-Sender mit Sprengstoffgü...,Panorama


## Fasttext

In [4]:
from gensim.models import KeyedVectors
from gensim.parsing.preprocessing import preprocess_string, strip_punctuation

In [5]:
embeddings = KeyedVectors.load_word2vec_format('./models/wiki.de.align.vec')

In [6]:
stopwords = ["der","die","das","den","des","dem"]

In [7]:
def tokenize(text):
    """Tokenizes the provided text
    Args:
        text (str): The text to be tokenized
    Returns:
        list(tuple(str, int)): A list of (token, count) pairs from the text without the stopwords.
    """

    # make everything lowercase and strip punctuation
    CUSTOM_FILTERS = [lambda x: x.lower(), strip_punctuation]
    tokens = preprocess_string(text, CUSTOM_FILTERS)

    # filter out all stopwords
    filtered_tokens = [w for w in tokens if not w in stopwords]

    # return the filtered tokens
    return filtered_tokens


In [8]:
def text_embedding(text):
    """Create the text embedding
    Args:
        text (str): The text to be embedded
    Returns:
        list(float): The array of values representing the text embedding
    """

    # prepare the embedding placeholder
    embedding = np.zeros(embeddings.vector_size, dtype=np.float32)

    if text is None:
        # return the default embedding in a vanilla python object
        return embedding

    # get the text terms with frequencies
    tokens = tokenize(text)
    # iterate through the terms and count the number of terms
    count = 0
    for token in tokens:
        # sum all token embeddings of the vector
        if token in embeddings.vocab.keys():
            embedding += embeddings[token]
            count += 1

    if count == 0:
        # return the empty embedding list
        return embedding.tolist()

    # average the embedding
    embedding = embedding / count


    # return the embedding in vanilla python object
    return embedding

In [9]:
print("elena")
print(embeddings.vocab)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
embds=[]
for ind in train.index.values:
    embds.append(text_embedding(X_train.loc[ind]))

In [11]:
embds=np.stack( embds, axis=0 )

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression

In [18]:
clf = RandomForestClassifier(max_depth=15)
clf.fit(embds,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=15, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [19]:
f1_score(clf.predict(embds),y_train, average='weighted')

0.9985112061121956

In [20]:
embds_test=[]
for ind in test.index.values:
    embds_test.append(text_embedding(X_test.loc[ind]))
embds=np.stack( embds, axis=0 )

In [21]:
f1_score(clf.predict(embds_test),y_test, average='weighted')

0.8096601263256422

In [22]:
clf.score(embds_test, y_test)

0.8092147955872809

In [23]:
print(clf.predict(embds_test))

['International' 'Wirtschaft' 'Wirtschaft' ... 'Wirtschaft' 'Wirtschaft'
 'Web']


In [39]:
clf=LogisticRegression(C=1000)
clf.fit(embds,y_train)
f1_score(clf.predict(embds),y_train, average='weighted')

/home/elena-carica/anaconda3/envs/embeddings/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8554687254024944

In [40]:
f1_score(clf.predict(embds_test),y_test, average='weighted')

0.8492762019674748